In [2]:
using CobraTools
using JuMP
using Gurobi # use your favourite solver

### Import model

In [3]:
# E. coli model
modelpath = joinpath("..", "models", "iJO1366.json") 
model = CobraTools.readmodel(modelpath)

LoadError: UndefVarError: readmodel not defined

### FBA
Convenience function to run standard FBA

In [4]:
biomass_rxn = findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M") # set biomass objective function
fbasol = CobraTools.fba(model, biomass_rxn)

LoadError: UndefVarError: model not defined

### pFBA
Convenience function to run standard pFBA

In [5]:
biomass_rxn = findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M") # set biomass objective function
pfbasol = CobraTools.pfba(model, biomass_rxn)

LoadError: UndefVarError: model not defined

### Get atoms consumed or produced by the model (e.g. through biomass)
Substrates are converted into products and biomass. This reveals the atoms consumed or produced by the biomass function based on the solution of a constraint based problem's fluxes.

In [6]:
biomass_rxn = findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M")
pfbasol = CobraTools.pfba(model, biomass_rxn) # use pFBA solution as an example
ad = CobraTools.atom_exchange(pfbasol)

LoadError: UndefVarError: model not defined

### Expose JuMP's underlying model for more exotic analyses
Julia's JuMP package is really excellent - why not expose so that you can write more sophisticated optimization problems yourself? 

In [7]:
cbmodel, v, mb, ubs, lbs = CobraTools.CBM(model); # NB: don't display these JuMP models, they are very big and will cause your system to freeze.
set_optimizer(cbmodel, Gurobi.Optimizer)
set_optimizer_attribute(cbmodel, "OutputFlag", 0) # quiet
# v = flux variables
# mb = mass balance constraints
# ubs = upper bounds on the variables
# lbs = lower bounds on the variables
# cbmodel = the constraint based model that includes
# the variables (rxns), bounds etc. are all in the order they appear in model.rxns

LoadError: UndefVarError: model not defined

In [8]:
# Note:  findfirst returns the reaction object with the specified ID
# Note: getindex or Model[] returns the index of a specified reaction object
biomass_index = model[findfirst(model.rxns, "BIOMASS_Ec_iJO1366_WT_53p95M")] 
glucose_index = model[findfirst(model.rxns, "EX_glc__D_e")]
o2_index = model[findfirst(model.rxns, "EX_o2_e")]
atpm_index = model[findfirst(model.rxns, "ATPM")]

# run an anaerobic simulation
# note, jump normalizes constraints, use this helper function to make setting the bounds easy
CobraTools.set_bound(o2_index, ubs, lbs; ub=0.0, lb=0.0)

@objective(cbmodel, Max, v[biomass_index])

# Solve optimization problem
optimize!(cbmodel) 
status = termination_status(cbmodel) == MOI.OPTIMAL

LoadError: UndefVarError: model not defined

### Sample from a model using ACHR